# Probability operations

1. Basic Probability Operations:
   - Empirical probability calculation
   - Joint probability
   - Conditional probability
   - Bayes' theorem

2. Probability Distributions:
   - Normal distribution
   - Uniform distribution
   - Bernoulli and Binomial
   - Poisson distribution
   - Categorical and Multivariate

3. Advanced Probability Operations:
   - Entropy calculation
   - KL divergence
   - Cross entropy
   - Mutual information

4. Sampling Operations:
   - Random sampling
   - Importance sampling
   - Rejection sampling

5. Probabilistic Models:
   - Gaussian Mixture Models
   - Hidden Markov Models

6. Probability Metrics and Tests:
   - Confidence intervals
   - Likelihood ratio tests


In [1]:
import torch
import torch.distributions as dist
import math

In [ ]:
# ===== Basic Probability Operations =====
class BasicProbability:
    @staticmethod
    def calculate_empirical_probability(events, total_samples):
        """Calculate empirical probability of events"""
        return events.float() / total_samples

    @staticmethod
    def joint_probability(x, y):
        """Calculate joint probability P(X,Y)"""
        total = len(x)
        unique_pairs = torch.stack([x, y]).T
        joint_counts = torch.unique(unique_pairs, dim=0, return_counts=True)[1]
        return joint_counts.float() / total

    @staticmethod
    def conditional_probability(x, y):
        """Calculate conditional probability P(X|Y)"""
        total_y = torch.sum(y)
        joint = torch.sum((x == 1) & (y == 1))
        return joint.float() / total_y

    @staticmethod
    def bayes_theorem(prior, likelihood, evidence):
        """
        Implement Bayes' Theorem: P(A|B) = P(B|A) * P(A) / P(B)
        """
        return (likelihood * prior) / evidence


In [ ]:
# ===== Probability Distributions =====
class ProbabilityDistributions:
    def __init__(self):
        self.distributions = {}
        
    def normal_distribution(self, mu, sigma):
        """Create normal distribution"""
        return dist.Normal(mu, sigma)
    
    def uniform_distribution(self, low, high):
        """Create uniform distribution"""
        return dist.Uniform(low, high)
    
    def bernoulli_distribution(self, probs):
        """Create Bernoulli distribution"""
        return dist.Bernoulli(probs)
    
    def binomial_distribution(self, n, p):
        """Create binomial distribution"""
        return dist.Binomial(n, p)
    
    def poisson_distribution(self, rate):
        """Create Poisson distribution"""
        return dist.Poisson(rate)
    
    def categorical_distribution(self, probs):
        """Create categorical distribution"""
        return dist.Categorical(probs)
    
    def multivariate_normal(self, mean, covariance):
        """Create multivariate normal distribution"""
        return dist.MultivariateNormal(mean, covariance)

In [ ]:
# ===== Advanced Probability Operations =====
class AdvancedProbability:
    @staticmethod
    def entropy(probs):
        """Calculate Shannon entropy"""
        return -torch.sum(probs * torch.log2(probs + 1e-10))
    
    @staticmethod
    def kl_divergence(p, q):
        """Calculate Kullback-Leibler divergence"""
        return torch.sum(p * (torch.log(p + 1e-10) - torch.log(q + 1e-10)))
    
    @staticmethod
    def cross_entropy(p, q):
        """Calculate cross entropy"""
        return -torch.sum(p * torch.log(q + 1e-10))
    
    @staticmethod
    def mutual_information(joint_probs, marginal_x, marginal_y):
        """Calculate mutual information"""
        log_ratio = torch.log(joint_probs / (marginal_x.unsqueeze(1) * marginal_y.unsqueeze(0)) + 1e-10)
        return torch.sum(joint_probs * log_ratio)


In [ ]:
# ===== Sampling Operations =====
class SamplingOperations:
    @staticmethod
    def random_sample(distribution, sample_size):
        """Generate random samples from distribution"""
        return distribution.sample((sample_size,))
    
    @staticmethod
    def importance_sampling(target_dist, proposal_dist, num_samples):
        """Perform importance sampling"""
        samples = proposal_dist.sample((num_samples,))
        log_weights = target_dist.log_prob(samples) - proposal_dist.log_prob(samples)
        weights = torch.exp(log_weights)
        normalized_weights = weights / weights.sum()
        return samples, normalized_weights
    
    @staticmethod
    def rejection_sampling(target_dist, proposal_dist, M, num_samples):
        """Perform rejection sampling"""
        accepted_samples = []
        while len(accepted_samples) < num_samples:
            proposed = proposal_dist.sample()
            ratio = target_dist.log_prob(proposed).exp() / (M * proposal_dist.log_prob(proposed).exp())
            if torch.rand(1) < ratio:
                accepted_samples.append(proposed)
        return torch.stack(accepted_samples)


In [ ]:
# ===== Probabilistic Models =====
class ProbabilisticModels:
    @staticmethod
    def gaussian_mixture_model(x, means, covs, weights):
        """
        Evaluate Gaussian Mixture Model probability
        """
        num_components = len(means)
        probs = torch.zeros(len(x))
        
        for i in range(num_components):
            distribution = dist.MultivariateNormal(means[i], covs[i])
            probs += weights[i] * torch.exp(distribution.log_prob(x))
            
        return probs
    
    @staticmethod
    def hidden_markov_model(observations, transition_matrix, emission_matrix, initial_probs):
        """
        Forward algorithm for Hidden Markov Model
        """
        num_states = len(initial_probs)
        num_observations = len(observations)
        
        # Forward matrix
        forward = torch.zeros(num_observations, num_states)
        
        # Initialize first observation
        forward[0] = initial_probs * emission_matrix[:, observations[0]]
        
        # Forward pass
        for t in range(1, num_observations):
            for s in range(num_states):
                forward[t, s] = torch.sum(forward[t-1] * transition_matrix[:, s]) * \
                              emission_matrix[s, observations[t]]
                              
        return forward

In [ ]:
# ===== Probability Metrics and Tests =====
class ProbabilityMetrics:
    @staticmethod
    def compute_confidence_interval(data, confidence=0.95):
        """
        Compute confidence interval for mean
        """
        n = len(data)
        mean = torch.mean(data)
        std_err = torch.std(data, unbiased=True) / torch.sqrt(torch.tensor(n).float())
        
        # Using normal approximation
        z_score = torch.tensor(1.96)  # 95% confidence
        margin_error = z_score * std_err
        
        return {
            'mean': mean,
            'confidence_interval': (mean - margin_error, mean + margin_error),
            'confidence_level': confidence
        }
    
    @staticmethod
    def likelihood_ratio_test(null_ll, alt_ll, df):
        """
        Perform likelihood ratio test
        """
        lr_statistic = -2 * (null_ll - alt_ll)
        # Chi-square test statistic
        return {
            'test_statistic': lr_statistic,
            'degrees_of_freedom': df
        }


In [ ]:
# ===== Example Usage =====
def demonstrate_probability_operations():
    # Basic probability example
    events = torch.tensor([1, 1, 0, 1, 0])
    prob = BasicProbability.calculate_empirical_probability(events.sum(), len(events))
    print(f"Empirical probability: {prob}")
    
    # Distribution example
    dist_ops = ProbabilityDistributions()
    normal_dist = dist_ops.normal_distribution(torch.tensor([0.0]), torch.tensor([1.0]))
    samples = normal_dist.sample((1000,))
    
    # Advanced probability example
    probs = torch.tensor([0.3, 0.4, 0.3])
    entropy = AdvancedProbability.entropy(probs)
    print(f"Entropy: {entropy}")
    
    # Sampling example
    proposal_dist = dist.Normal(0, 1)
    target_dist = dist.Normal(1, 0.5)
    samples, weights = SamplingOperations.importance_sampling(target_dist, proposal_dist, 1000)
    
    return {
        'empirical_prob': prob,
        'samples': samples,
        'entropy': entropy,
        'importance_weights': weights
    }